# Seattle Open Data Portal for Data Analytics on Seattle

## Queries different Seattle data repositories to get the latest news on health

In [41]:
# import libraries
import requests
import urllib
import webbrowser
import json
import sys
import numpy
import matplotlib.pyplot as plt
import hvplot.pandas 
import sklearn
from dotenv import load_dotenv
import pandas as pd
import time
import datetime
import geopandas as gpd

from socrata.authorization import Authorization
from socrata import Socrata
import os
import argparse

In [42]:
# Load environment variables
load_dotenv()

True

In [43]:
# retrieve environment email and password
socrata_email = os.getenv("SOCRATA_EMAIL")
socrata_password = os.getenv("SOCRATA_PASSWORD")

# Check is loaded into variables
display(type(socrata_email))
display(type(socrata_password))

str

str

In [44]:
auth = Authorization(
    "data.seattle.gov",
    socrata_email,
    socrata_password
)

In [45]:
# create an HTTP Basic request with endpoint
# Data for crime in seattle based on location
data_url = 'https://data.seattle.gov/resource/kzjm-xkqj.json'
response = requests.get(data_url, auth=(socrata_email, socrata_password))

In [46]:
# turn response to json object
response.json()[0]

{'address': '1700 17th Ave',
 'type': 'Nurseline/AMR',
 'datetime': '2023-02-07T04:24:00.000',
 'latitude': '47.616443',
 'longitude': '-122.310151',
 'report_location': {'type': 'Point', 'coordinates': [-122.310151, 47.616443]},
 'incident_number': 'F230016112',
 ':@computed_region_ru88_fbhk': '9',
 ':@computed_region_kuhn_3gp2': '4',
 ':@computed_region_q256_3sug': '19578'}

In [47]:
# design a dataframe with just the datetime, type, latitude and longitude
test_list = [
    response.json()[0]['datetime'],
    response.json()[0]['type'],
    response.json()[0]['latitude'],
    response.json()[0]['longitude']
]

test_list

['2023-02-07T04:24:00.000', 'Nurseline/AMR', '47.616443', '-122.310151']

In [48]:
# put data into dataframe
crime_df = pd.DataFrame.from_records(response.json())

# set datetime as index
crime_df = crime_df.set_index('datetime')
display(crime_df.head(10))
display(crime_df.tail(10))

,address,type,latitude,longitude,report_location,incident_number,:@computed_region_ru88_fbhk,:@computed_region_kuhn_3gp2,:@computed_region_q256_3sug,:@computed_region_2day_rhn5,:@computed_region_cyqu_gs94
datetime,,,,,,,,,,,
2023-02-07T04:24:00.000,1700 17th Ave,Nurseline/AMR,47.616443,-122.310151,"{'type': 'Point', 'coordinates': [-122.310151,...",F230016112,9,4,19578,NaN,NaN
2023-02-07T04:04:00.000,Lenora St / 6th Ave,Aid Response,47.61516,-122.339614,"{'type': 'Point', 'coordinates': [-122.339614,...",F230016110,56,7,19576,NaN,NaN
2023-02-07T03:22:00.000,3rd Ave / James St,Aid Response,47.602769,-122.331409,"{'type': 'Point', 'coordinates': [-122.331409,...",F230016109,14,19,18379,NaN,NaN
2023-02-07T03:20:00.000,503 S Royal Brougham Way,Aid Response,47.592399,-122.327256,"{'type': 'Point', 'coordinates': [-122.327256,...",F230016108,27,22,19584,NaN,NaN
2023-02-07T03:17:00.000,308 4th Ave S,Medic Response,47.599968,-122.328965,"{'type': 'Point', 'coordinates': [-122.328965,...",F230016107,27,22,18379,NaN,NaN
2023-02-07T03:12:00.000,600 4th Ave,Aid Response,47.603233,-122.330297,"{'type': 'Point', 'coordinates': [-122.330297,...",F230016106,14,19,18379,NaN,NaN
2023-02-07T03:11:00.000,710 Cherry St,Aid Response,47.605387,-122.327511,"{'type': 'Point', 'coordinates': [-122.327511,...",F230016105,19,12,18379,NaN,NaN
2023-02-07T03:07:00.000,5th Ave S / S Royal Brougham Way,Aid Response,47.592394,-122.327395,"{'type': 'Point', 'coordinates': [-122.327395,...",F230016103,27,22,19584,NaN,NaN
2023-02-07T03:07:00.000,603 22nd Ave E,Low Acuity Response,47.624415,-122.303551,"{'type': 'Point', 'coordinates': [-122.303551,...",F230016104,35,6,18789,NaN,NaN


,address,type,latitude,longitude,report_location,incident_number,:@computed_region_ru88_fbhk,:@computed_region_kuhn_3gp2,:@computed_region_q256_3sug,:@computed_region_2day_rhn5,:@computed_region_cyqu_gs94
datetime,,,,,,,,,,,
2023-02-04T01:13:00.000,1039 6th Ave S,Aid Response,47.593751,-122.325968,"{'type': 'Point', 'coordinates': [-122.325968,...",F230014819,27,22,19584,NaN,NaN
2023-02-04T01:08:00.000,E Harrison St / Belmont Ave E,Aid Response,47.62199,-122.324024,"{'type': 'Point', 'coordinates': [-122.324024,...",F230014818,8,5,18376,NaN,NaN
2023-02-04T00:55:00.000,306 23rd Ave S,MVI - Motor Vehicle Incident,47.600257,-122.302236,"{'type': 'Point', 'coordinates': [-122.302236,...",F230014816,32,18,17919,NaN,NaN
2023-02-04T00:45:00.000,609 Yesler Way,Aid Response,47.601709,-122.326189,"{'type': 'Point', 'coordinates': [-122.326189,...",F230014815,14,22,18379,NaN,NaN
2023-02-04T00:32:00.000,DENNY WAY / STEWART ST,Triaged Incident,47.618492,-122.330723,"{'type': 'Point', 'coordinates': [-122.330723,...",F230014814,56,10,18081,NaN,NaN
2023-02-04T00:30:00.000,707 S Henderson St,Nurseline/AMR,47.523073,-122.325453,"{'type': 'Point', 'coordinates': [-122.325453,...",F230014812,59,15,18388,NaN,NaN
2023-02-04T00:29:00.000,6th Ave S / S Royal Brougham Way,Aid Response,47.592393,-122.325968,"{'type': 'Point', 'coordinates': [-122.325968,...",F230014811,27,22,19584,NaN,NaN
2023-02-04T00:28:00.000,Denny Way / Stewart St,MVI - Motor Vehicle Incident,47.618492,-122.330723,"{'type': 'Point', 'coordinates': [-122.330723,...",F230014810,56,10,18081,NaN,NaN
2023-02-04T00:25:00.000,Ne 47th St / 18th Ave Ne,Aid Response,47.663058,-122.308354,"{'type': 'Point', 'coordinates': [-122.308354,...",F230014808,60,47,18383,NaN,NaN


In [49]:
# trim dataframe to just type, latitude and longitude
trimmed_crime_df = crime_df.loc[:, ['type', 'latitude', 'longitude']]
trimmed_crime_df.columns = ['type', 'Latitude', 'Longitude']
trimmed_crime_df


,type,Latitude,Longitude
datetime,,,
2023-02-07T04:24:00.000,Nurseline/AMR,47.616443,-122.310151
2023-02-07T04:04:00.000,Aid Response,47.61516,-122.339614
2023-02-07T03:22:00.000,Aid Response,47.602769,-122.331409
2023-02-07T03:20:00.000,Aid Response,47.592399,-122.327256
2023-02-07T03:17:00.000,Medic Response,47.599968,-122.328965
...,...,...,...
2023-02-04T00:30:00.000,Nurseline/AMR,47.523073,-122.325453
2023-02-04T00:29:00.000,Aid Response,47.592393,-122.325968
2023-02-04T00:28:00.000,MVI - Motor Vehicle Incident,47.618492,-122.330723


In [50]:
# investigate dataframe
trimmed_crime_df.dtypes

type         object
Latitude     object
Longitude    object
dtype: object

In [51]:
# create ordered by date to see date span


In [52]:
#Change latitude and longitude to floats
# rename dataframe for easy use
plotting_df = trimmed_crime_df
plotting_df['Latitude'] = plotting_df['Latitude'].astype('float')
plotting_df['Longitude'] = plotting_df['Longitude'].astype('float')
plotting_df.dtypes

type          object
Latitude     float64
Longitude    float64
dtype: object

In [53]:
# check unique values
print(trimmed_crime_df['type'].unique())

['Nurseline/AMR' 'Aid Response' 'Medic Response' 'Low Acuity Response'
 'Alarm Bell' 'Auto Fire Alarm' 'Automatic Fire Alarm Resd'
 'Investigate Out Of Service' 'Triaged Incident' 'Encampment Fire'
 'MVI - Motor Vehicle Incident' 'Dumpster Fire' 'Medic Response- Overdose'
 'Rubbish Fire' 'Illegal Burn' 'Fire in Building' 'Unk Odor'
 'Rescue Lock In/Out' 'Food On The Stove' 'Rescue Elevator' 'Car Fire'
 '1RED 1 Unit' 'Medic Response- 7 per Rule' 'AFA4 - Auto Alarm 2 + 1 + 1'
 'Electrical Problem' 'MVI Freeway' '4RED - 2 + 1 + 1' 'Trans to AMR'
 'Crisis Center' 'Aid Response Yellow' 'EVENT - Special Event'
 'Hang-Up- Aid' 'Water Job Minor' 'Scenes Of Violence 7' '2RED - 1 + 1'
 'Scenes Of Violence Aid' 'Medic Response- 6 per Rule'
 'Activated CO Detector' 'Water Rescue Response' 'Natural Gas Leak'
 'Car Fire Freeway' 'Fuel Spill' 'Automatic Fire Alarm False'
 'Rescue Extrication' 'Chimney Fire' 'Natural Gas Odor' 'Rescue Standby'
 'Brush Fire']


In [54]:
# drop duplicates
no_duplicate_df = trimmed_crime_df[~trimmed_crime_df.index.duplicated()]
no_duplicate_df = no_duplicate_df.reset_index()
no_duplicate_df


,datetime,type,Latitude,Longitude
0,2023-02-07T04:24:00.000,Nurseline/AMR,47.616443,-122.310151
1,2023-02-07T04:04:00.000,Aid Response,47.615160,-122.339614
2,2023-02-07T03:22:00.000,Aid Response,47.602769,-122.331409
3,2023-02-07T03:20:00.000,Aid Response,47.592399,-122.327256
4,2023-02-07T03:17:00.000,Medic Response,47.599968,-122.328965
...,...,...,...,...
891,2023-02-04T00:30:00.000,Nurseline/AMR,47.523073,-122.325453
892,2023-02-04T00:29:00.000,Aid Response,47.592393,-122.325968
893,2023-02-04T00:28:00.000,MVI - Motor Vehicle Incident,47.618492,-122.330723
894,2023-02-04T00:25:00.000,Aid Response,47.663058,-122.308354


In [55]:
# get value counts of types
value_counts = no_duplicate_df['type'].value_counts()
value_counts.dtype

dtype('int64')

In [56]:
# export value counts to an external file for display purposes
value_counts.to_csv('./data/value_counts_crimes.csv')

In [57]:
# plot geospatially
geo_plot = plotting_df.hvplot.points(
    'Longitude', 
    'Latitude', 
    geo=True, 
    color='type',
    size=50,
    alpha=0.8,
    tiles='OSM',
    frame_width = 700,
    frame_height = 1500,
    title='911 Calls from 02-04-2023 - 02-07-2023'
    )

geo_plot


c:\Users\silva\anaconda3\envs\dev\lib\site-packages\cartopy\crs.py:245: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  if len(multi_line_string) > 1:
c:\Users\silva\anaconda3\envs\dev\lib\site-packages\cartopy\crs.py:297: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for line in multi_line_string:
c:\Users\silva\anaconda3\envs\dev\lib\site-packages\cartopy\crs.py:364: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  if len(p_mline) > 0:
c:\Users\silva\anaconda3\envs\dev\lib\site-packages\cartopy\crs.py:245

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Longitude,Latitude]   (type)

In [58]:
# plot geospatially with options
geo_plot_grouped = plotting_df.hvplot.points(
    'Longitude', 
    'Latitude', 
    geo=True, 
    size=50,
    color='type',
    alpha=0.8,
    tiles='OSM',
    frame_width = 700,
    frame_height = 500,
    title='911 Calls from 02-04-2023 - 02-07-2023'
    )

geo_plot_grouped

c:\Users\silva\anaconda3\envs\dev\lib\site-packages\cartopy\crs.py:245: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  if len(multi_line_string) > 1:
c:\Users\silva\anaconda3\envs\dev\lib\site-packages\cartopy\crs.py:297: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for line in multi_line_string:
c:\Users\silva\anaconda3\envs\dev\lib\site-packages\cartopy\crs.py:364: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  if len(p_mline) > 0:
c:\Users\silva\anaconda3\envs\dev\lib\site-packages\cartopy\crs.py:245

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Longitude,Latitude]   (type)

In [62]:
# create chart from csv of value counts for display

# read saved csv with new headers
value_counts_df = pd.read_csv('./data/value_counts_crimes.csv', header=None)

columns=['Type of Crime', 'Occurrences']

value_counts_df.columns=columns
value_counts_df = value_counts_df.dropna()
value_counts_df

,Type of Crime,Occurrences
1,Aid Response,461
2,Medic Response,75
3,Auto Fire Alarm,53
4,Nurseline/AMR,31
5,Triaged Incident,30
6,Low Acuity Response,30
7,MVI - Motor Vehicle Incident,23
8,Medic Response- 7 per Rule,18
9,Investigate Out Of Service,16
10,Rescue Elevator,16


In [63]:
# turn value counts df to a csv
value_counts_df.to_csv('./images/table_counts.csv')

In [64]:
# create type of crime as index
new_df = value_counts_df.set_index('Type of Crime')
new_df = new_df.dropna()
new_df


,Occurrences
Type of Crime,
Aid Response,461
Medic Response,75
Auto Fire Alarm,53
Nurseline/AMR,31
Triaged Incident,30
Low Acuity Response,30
MVI - Motor Vehicle Incident,23
Medic Response- 7 per Rule,18
Investigate Out Of Service,16


In [65]:
new_df.to_csv('./images/table_counts.csv')

In [61]:
# 